In [3]:
import numpy as np
import pandas as pd
from tools import generate_samples, data_generator
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [14]:
def create_model():
    nn = Sequential()
    nn.add(Dense(400, input_shape=(400,), activation='relu'))
    nn.add(Dense(400, activation='relu'))
    nn.add(Dense(400, activation='sigmoid'))
    nn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy', 'mae'])
    return nn

In [15]:
es = EarlyStopping(monitor='loss', patience=6, min_delta=0.01)

In [16]:
train_df = pd.read_csv("resources/train.csv", index_col=0)
test_df = pd.read_csv("resources/test.csv", index_col=0)

In [17]:
models = dict()
for delta in range(1, 6):
    models[delta] = create_model()
    models[delta].fit_generator(data_generator(delta=delta), steps_per_epoch=10, epochs=50, verbose=2, callbacks=[es])
    delta_df = train_df[train_df.delta == delta]
    X, Y = delta_df.iloc[:, 1:401], delta_df.iloc[:, 401:]
    models[delta].fit(X, Y, epochs=15, verbose=2, batch_size=32, callbacks=[es])

Epoch 1/50
 - 6s - loss: 0.5777 - acc: 0.7385 - mean_absolute_error: 0.3895
Epoch 2/50
 - 2s - loss: 0.5416 - acc: 0.7750 - mean_absolute_error: 0.3558
Epoch 3/50
 - 3s - loss: 0.5420 - acc: 0.7733 - mean_absolute_error: 0.3578
Epoch 4/50
 - 3s - loss: 0.5392 - acc: 0.7725 - mean_absolute_error: 0.3532
Epoch 5/50
 - 3s - loss: 0.5320 - acc: 0.7776 - mean_absolute_error: 0.3506
Epoch 6/50
 - 3s - loss: 0.5320 - acc: 0.7728 - mean_absolute_error: 0.3517
Epoch 7/50
 - 3s - loss: 0.5280 - acc: 0.7752 - mean_absolute_error: 0.3468
Epoch 8/50
 - 3s - loss: 0.5278 - acc: 0.7721 - mean_absolute_error: 0.3475
Epoch 9/50
 - 3s - loss: 0.5169 - acc: 0.7777 - mean_absolute_error: 0.3393
Epoch 10/50
 - 4s - loss: 0.5172 - acc: 0.7773 - mean_absolute_error: 0.3387
Epoch 11/50
 - 3s - loss: 0.5171 - acc: 0.7745 - mean_absolute_error: 0.3402
Epoch 12/50
 - 4s - loss: 0.5166 - acc: 0.7737 - mean_absolute_error: 0.3410
Epoch 13/50
 - 3s - loss: 0.5067 - acc: 0.7771 - mean_absolute_error: 0.3325
Epoch 14

In [20]:
submit_df = pd.DataFrame(index=test_df.index, columns=['start.' + str(_) for _ in range(1, 401)])

In [26]:
# submit_df[test_df.delta == 1] = models[1].predict(test_df[test_df.delta == 1])
for delta in range(1, 6):
    submit_df[test_df.delta == delta] = models[delta].predict(test_df[test_df.delta == delta].iloc[:, 1:]).round(0)

In [28]:
submit_df.to_csv('nn_submit.csv')